# Team Members:
- Amirhossein Mobayen

## Topic:
AI Chess Master

# Objective:
Build a Vision AI which understands a position by looking at the board!

In [28]:
import os

# Basic packages
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.optimizers import SGD

from tensorflow.keras import mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)


# Step 1: Dataset Preparation


In [29]:
# Download the dataset from Kaggle and extract the relevant files.
# URL: https://www.kaggle.com/koryakinp/chess-positions

# Define the input image size for the model
img_height, img_width = 224, 224
batch_size = 32

# Define the path to the dataset directory
dataset_dir = './dataset/'

# Step 2: Data Preprocessing

In [30]:
# Load the dataset without setting label_mode
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    seed=123,
    color_mode='grayscale',
    image_size=(img_height, img_width),
    batch_size=batch_size,
    validation_split=0.2,
    subset='training'
)

# Extract file names as labels
train_labels = []
for image_batch, label_batch in train_ds:
    label_batch = [str(label) for label in label_batch]  # Convert label batch to strings
    train_labels.extend([os.path.basename(file_path) for file_path in label_batch])

# Convert labels to NumPy array
train_labels = np.array(train_labels)


Found 100000 files belonging to 2 classes.
Using 80000 files for training.


2023-07-15 18:58:58.886441: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [80000]
	 [[{{node Placeholder/_4}}]]
2023-07-15 18:58:58.886647: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [80000]
	 [[{{node Placeholder/_4}}]]


In [31]:
# Split the train_ds dataset into training and validation subsets
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    seed=123,
    color_mode='grayscale',
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False,
    validation_split=0.2,
    subset='validation'
)

# Extract file names as labels for val_ds
val_labels = []
for _, label_batch in val_ds:
    label_batch = [str(label.numpy()) for label in label_batch]  # Convert label batch to strings
    val_labels.extend(label_batch)

# Convert labels to NumPy array
val_labels = np.array(val_labels)

Found 100000 files belonging to 2 classes.
Using 20000 files for validation.


2023-07-15 18:59:12.922294: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [20000]
	 [[{{node Placeholder/_4}}]]
2023-07-15 18:59:12.922464: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [20000]
	 [[{{node Placeholder/_4}}]]


# Step 3: Model Selection and Architecture

In [32]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='softmax')  # Adjust num_classes based on your dataset
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 222, 222, 32)      320       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 394272)            0         
                                                                 
 dense_2 (Dense)             (None, 64)                25233472  
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 25,233,857
Trainable params: 25,233,857
Non-trainable params: 0
__________________________________________

# Step 4: Model Training

In [33]:
# Assuming you have a separate validation dataset called val_ds
model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_ds, batch_size=32, epochs=10, validation_data=val_ds)


Epoch 1/10


/home/amir/anaconda3/envs/tf/lib/python3.10/site-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


   9/2500 [..............................] - ETA: 3:48:38 - loss: nan - accuracy: 0.5243

KeyboardInterrupt: 

# Step 5: Model Evaluation

In [ ]:
# Evaluate the model on the testing dataset
test_loss, test_accuracy = model.evaluate(val_ds)

In [ ]:
def plot_results(history):
    hist_df = pd.DataFrame(history.history)
    hist_df.columns = ["loss", "accuracy", "val_loss", "val_accuracy"]
    hist_df.index = np.arange(1, len(hist_df) + 1)

    fig, axs = plt.subplots(nrows=2, sharex=True, figsize=(16, 10))
    axs[0].plot(hist_df.val_accuracy, lw=3, label='Validation Accuracy')
    axs[0].plot(hist_df.accuracy, lw=3, label='Training Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].grid()
    axs[0].legend(loc=0)
    axs[1].plot(hist_df.val_loss, lw=3, label='Validation Loss')
    axs[1].plot(hist_df.loss, lw=3, label='Training Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].grid()
    axs[1].legend(loc=0)

    plt.show()


plot_results(history)

# Step 6: Model Deployment

In [ ]:
# Save the trained model weights for future use
model.save('chessboard_model.h5')

# Load the saved model
loaded_model = tf.keras.models.load_model('chessboard_model.h5')

In [ ]:
def convert_predictions_to_fen(predictions):
    fen_positions = []
    for prediction in predictions:
        fen = ""
        board = prediction.argmax(axis=-1)

        # Convert the board array into FEN format
        for row in range(8):
            empty_counter = 0
            for col in range(8):
                piece = board[row * 8 + col]
                if piece == 0:
                    empty_counter += 1
                else:
                    if empty_counter > 0:
                        fen += str(empty_counter)
                        empty_counter = 0
                    fen += chr(piece + 96)  # Convert piece value to ASCII character
            if empty_counter > 0:
                fen += str(empty_counter)
            if row < 7:
                fen += "/"

        # Append the FEN position to the list
        fen_positions.append(fen)

    return fen_positions

In [ ]:
# Iterate over a few samples from the validation dataset
for images, labels in val_ds.take(5):
    # Make predictions on the images
    predictions = loaded_model.predict(images)

    # Convert predictions to FEN format
    predicted_positions = convert_predictions_to_fen(predictions)

    # Print the true label and predicted position for each sample
    for true_label, predicted_position in zip(labels, predicted_positions):
        print("True Label: ", true_label)
        print("Predicted Position: ", predicted_position)
        print("---")
